In [280]:
import sys 
sys.path.append('../../')
from src.import_packages import *

from bs4 import BeautifulSoup
import requests

import re

### Síterepek listája

In [285]:
url = 'https://sielok.hu/sitereplista/orszag/ausztria/listadizajn/rendabcnov/'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [286]:
siterepek = {}

for i in soup.find_all('tr'):
    _td = i.find_all('td')
    if (len(_td) != 0) and ('siterep' in _td[2].find('a')['href']):
        siterepek[_td[2].text] = 'https://sielok.hu' + _td[2].find('a')['href']

In [287]:
len(siterepek)

252

### Síterep infók

In [288]:
def siterep_infok(nev):

    i_ = siterepek[nev]
    page = requests.get(i_)
    soup = BeautifulSoup(page.text) 
    
    alap_adatok = soup.find_all('table', class_ = 'tedatable')[0]
    alap_adatok_dict = {}

    for elem in alap_adatok.tbody.find_all('tr'):
        alap_adatok_dict[elem.find('td', class_ = 'tedacol1').text] = elem.find('td', class_ = 'tedacol2').text
        
        
    szin_adatok = soup.find_all('div', class_ = 'terepSzinek2')[0]
    szin_adatok_dict = {}
    szin_adatok_key_map = {'ski_blue.svg': 'Kék', 'ski_red.svg': 'Piros', 'ski_black.svg': 'Fekete'}

    for szin, hossz in zip(szin_adatok.find_all('img'), szin_adatok.find_all('div', class_ = 'terepSzinLabel')):
        szin_adatok_dict[szin['src'].split('/')[-1]] =  float(re.sub("[^0-9.]", "", hossz.text.replace(',', '.').strip()))

    szin_adatok_dict = {szin_adatok_key_map[k]:szin_adatok_dict[k] for k in szin_adatok_dict }


    lift_adatok = soup.find_all('div', class_ = 'terepLiftPart')
    lift_adatok_dict = {}

    for lift in lift_adatok:
        lift_adatok_dict[lift.find('img')['title']] = int((lift.text.strip()))

    egyeb_adatok = soup.find_all('div', class_ = 'terepInfoPartWrap')
    egyeb_adatok_lista = []

    for i in egyeb_adatok:
        egyeb_adatok_lista.append(i.text.strip().replace('\xa0', ' '))

    szezoneleje = np.nan
    szezonveg = np.nan
    tavolsag = np.nan
    napijegy = np.nan
        
    for i in egyeb_adatok_lista:   

        try: 
            if 'időszak' in i:
                idoszak = i.replace('Síszezon időszaka: ', '').strip()
                szezonveg = idoszak.split('–')[1].strip()
                szezoneleje = idoszak.split('–')[0].strip()

            if 'Budapest' in i:
                tavolsag = float(i.replace('Budapest:', '').split('km')[0].strip())

            if 'naponta változik' in i:
                napijegy = i.replace('\n', '').replace('EUR', '€').strip()
                napijegy = float(napijegy.split('€')[0].replace('A síbérlet ára naponta változik, napijegy max ', '').strip())

            elif 'napijegy' in i:
                napijegy = i.replace('\n', '').replace('EUR', '€').strip()
                napijegy = float(napijegy.split('€')[0].replace('Felnőtt napijegy: ', '').strip())

        except:
            continue

    adatok = {'Síterep' : nev,
    'Link' : i_,
    'Régió' : soup.find_all('span', class_ = 'navbarItem')[-1].text.strip(),
    'Parkoló' : int(alap_adatok_dict['Magasság'].split('-')[0].strip()),
    'Hegytető' : int(re.sub("[^0-9]", "", alap_adatok_dict['Magasság'].split('-')[1].strip())),
    'Sípályák hossza' : float(re.sub("[^0-9.]", "", alap_adatok_dict['Sípályák hossza'].replace(',', '.').strip())),
    'Szintkülönbség' : int(re.sub("[^0-9]", "", alap_adatok_dict['Szintkülönbség'].strip())),
    'Liftek száma' : int(re.sub("[^0-9]", "", alap_adatok_dict['Liftek száma'].replace(',', '.').strip())),
    'Szezon eleje' : szezoneleje,
    'Szezon vége' : szezonveg,
    'Távolság Budapesttől' : tavolsag,
    'Felnőtt napijegy ára' : napijegy}

    adatok.update(lift_adatok_dict)
    adatok.update(szin_adatok_dict)

    return adatok

In [289]:
%%time

kihagyhato = []
collector = []

for i in siterepek.keys():
    try:
        collector.append(siterep_infok(i))
    except:
        kihagyhato.append(siterepek[i])

Wall time: 2min 20s


In [290]:
print(len(collector))
print(len(kihagyhato))

251
1


### Concat

In [291]:
data = pd.DataFrame(collector)
print(data.shape)

(251, 21)


In [292]:
data['Liftek száma'] = data['Liftek száma'].astype(int)

for i in data.filter(like = 'lift').columns:
    data[i].fillna(0, inplace = True)
    data[i] = data[i].astype(int)

data['Vonat'].fillna(0, inplace = True)
data['Vonat'] = data['Vonat'].astype(int)

data['Kék'].fillna(0, inplace = True)
data['Piros'].fillna(0, inplace = True)
data['Fekete'].fillna(0, inplace = True)

In [293]:
data.isnull().sum()

Síterep                  0
Link                     0
Régió                    0
Parkoló                  0
Hegytető                 0
Sípályák hossza          0
Szintkülönbség           0
Liftek száma             0
Szezon eleje            67
Szezon vége             67
Távolság Budapesttől     3
Felnőtt napijegy ára    32
Húzólift                 0
Kombilift                0
Kék                      0
Piros                    0
Fekete                   0
Ülőlift                  0
Nagykabinos lift         0
Kiskabinos lift          0
Vonat                    0
dtype: int64

In [295]:
for i in ['Sípályák hossza', 'Kék', 'Piros', 'Fekete']:
    data.loc[((data['Sípályák hossza'] > 200) & (~data['Síterep'].isin(['Ischgl - Silvretta Arena', 
                                                                        'Saalbach Hinterglemm Leogang Fieberbrunn',
                                                                        'Ski Arlberg - St. Anton - Lech - Zürs - Warth',
                                                                        'SkiWelt Wilder Kaiser - Brixental']))), i] = data.loc[data['Sípályák hossza'] > 302, i] / 1000
                                                                        
data.loc[data['Kék'] > data['Sípályák hossza'], 'Kék'] = data.loc[data['Kék'] > data['Sípályák hossza'], 'Kék'] / 1000
data.loc[data['Piros'] > data['Sípályák hossza'], 'Piros'] = data.loc[data['Piros'] > data['Sípályák hossza'], 'Piros'] / 1000
data.loc[data['Fekete'] > data['Sípályák hossza'], 'Fekete'] = data.loc[data['Fekete'] > data['Sípályák hossza'], 'Fekete'] / 1000

In [296]:
data = data[data['Link'] != 'https://sielok.hu/siterep/wiesen/']
data = data[data['Régió'] != 'Ausztria']

In [297]:
data.describe(percentiles = [.01, .1, .9, .99]).T

,count,mean,std,min,1%,10%,50%,90%,99%,max
Parkoló,249.0,1036.698795,347.184124,421.0,477.32,666.4,980.0,1542.00,2129.840,2685.0
Hegytető,249.0,1747.100402,561.031483,690.0,767.76,1074.4,1743.0,2490.40,3231.760,3440.0
Sípályák hossza,249.0,27.948594,43.968401,0.4,0.50,2.0,12.5,65.92,241.600,302.0
Szintkülönbség,249.0,710.401606,476.104716,37.0,48.96,161.6,622.0,1358.00,1896.000,2261.0
Liftek száma,249.0,9.293173,12.326146,1.0,1.00,2.0,5.0,20.20,63.760,89.0
Távolság Budapesttől,246.0,594.353659,176.528245,291.0,298.45,344.5,587.0,838.50,886.550,896.0
Húzólift,249.0,4.799197,4.609755,0.0,0.48,2.0,4.0,9.00,25.640,33.0
Kombilift,249.0,0.064257,0.261601,0.0,0.00,0.0,0.0,0.00,1.000,2.0
Kék,249.0,11.170281,18.600726,0.0,0.00,1.0,5.0,26.36,108.240,140.0
Piros,249.0,13.353815,21.779972,0.0,0.00,0.5,6.0,34.00,116.160,143.0


In [298]:
data.describe(include = 'O').T

,count,unique,top,freq
Síterep,249,249,Abtenau im Lammertal - Karkogel,1
Link,249,249,https://sielok.hu/siterep/abtenau/,1
Régió,249,18,Tirol,70
Szezon eleje,184,28,2022.12.17.,40
Szezon vége,184,20,2023.04.10.,47
Felnőtt napijegy ára,219.0,85.0,55.0,11.0


In [299]:
remap_regio = {'Schladming - Dachstein sírégió' : 'Stájerország', 'Gastein sírégió' : 'Salzburg', 'Lungau sírégió' : 'Salzburg', 
               'Seefeld sírégió' : 'Tirol', 'Lermoos sírégió' : 'Tirol', 'Zillertal 3000 sírégió' : 'Tirol', 'Zell am See - Kaprun sírégió' : 'Salzburg',
               'Lienz sírégió' : 'Tirol', 'Pitztal sírégió' : 'Tirol', 'Nauders sírégió' : 'Tirol', 'Berchtesgaden sírégió' : 'Salzburg'}

data['Régió'] = data['Régió'].replace(remap_regio)

In [300]:
data['Régió'].value_counts()

Tirol             82
Stájerország      48
Salzburg          34
Vorarlberg        24
Karintia          24
Alsó-Ausztria     22
Felső-Ausztria    15
Name: Régió, dtype: int64

In [301]:
data.to_csv('data/data.csv', index = False)